In [1]:
####################################
# Special code for Jupyter Notebook
####################################
# Show plots in the notebook

%matplotlib inline 
#%matplotlib notebook 

# Automatically reload any packages that changed
#%load_ext autoreload
#%autoreload 2

# Display multiple arguments
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

##################
# Import librarys
##################
# Third party libraries

import os
import sys
import glob
import json
import pandas as pd
import numpy as np
import pprint
import statistics as stat
import scipy as sp
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import rainflow
import OrcFxAPI


# Python libraries
module_path = r"C:\Users\snod\OneDrive\CloudStation\DTU\Master in Wind Energy\06 - Master Thesis\19 - python modules"
sys.path.append(module_path)

import stephan_py as st
import pywasp_edu as pw

############################################
# Limit number of rows to print from Pandas
############################################
pd.options.display.max_rows = 40

pi = np.arccos(-1)

# Used in order to embed figures in the text
from IPython.display import Image  

# Data extraction of time histories from Orcaflex

### Assumes that all *.sim files are of similar structure

In [2]:
# Definitions
folders = [r"D:\100 - Project files\02 - Orcaflex Runs\302 - IPC floating set 3 gain -10 sat 3deg"] #,
          # r"D:\100 - Project files\02 - Orcaflex Runs\202 - NTM Batch UMain contr WE_mode2 - IPC - 2" 
          #]

t_start_res = 0 # [s]
t_end_res = 600 # [s]
time_step = 0.1 # [s]
time_defs = [t_start_res, t_end_res, time_step]


In [3]:
# Read defined extractions from json file 
  
# Opening JSON file
with open('extraction_defs.json') as json_file:
    extraction_data = json.load(json_file)
    
pprint.pprint(extraction_data, sort_dicts=False, width=150)

{'RootMxb1': ['Root connection Ey moment', '15MW RWT', 'OrcFxAPI.oeTurbine(1)'],
 'RootMyb1': ['Root connection Ex moment', '15MW RWT', 'OrcFxAPI.oeTurbine(1)'],
 'RootMzb1': ['Root connection Ez moment', '15MW RWT', 'OrcFxAPI.oeTurbine(1)'],
 'TipDxb1': ['In plane deflection', '15MW RWT', 'OrcFxAPI.oeTurbineEndB(1)'],
 'TipDyb1': ['Out of plane deflection', '15MW RWT', 'OrcFxAPI.oeTurbineEndB(1)'],
 'NcIMUTAxs': ['GX acceleration', '15MW RWT', 'None'],
 'NcIMUTAys': ['GY acceleration', '15MW RWT', 'None'],
 'NcIMUTAzs': ['GZ acceleration', '15MW RWT', 'None'],
 'YawBrTDxt': ['Dynamic x', 'Tower', 'OrcFxAPI.oeEndA'],
 'YawBrTDyt': ['Dynamic y', 'Tower', 'OrcFxAPI.oeEndA'],
 'TwrBsMxt': ['x bend moment', 'Tower', 'OrcFxAPI.oeEndB'],
 'TwrBsMyt': ['y bend moment', 'Tower', 'OrcFxAPI.oeEndB'],
 'PtfmSurge': ['Primary X', 'Floater', 'OrcFxAPI.oeVessel(0.0, 0.0, 0.0)'],
 'PtfmHeave': ['Primary Z', 'Floater', 'OrcFxAPI.oeVessel(0.0, 0.0, 0.0)'],
 'PtfmPitch': ['Primary rotation 2', 'Floater'

In [4]:
# Insert extraction definition

if 0:
    extraction_data["TwrBsShy"]= ['y shear force', 'Tower', 'OrcFxAPI.oeEndB']

    pprint.pprint(extraction_data, sort_dicts=False, width=150)

In [5]:
# Remove extraction definition

if 0:
    extraction_data.pop('TwrBsShy', None)
    
    pprint.pprint(extraction_data, sort_dicts=False, width=150)

In [6]:
###################################################
# Test of extraction definitions                  #
###################################################

if 0:

    # Make list of *.sim files
    sim_files, sim_file_folders_list, prefix = st.folders_files.list_simfiles(folders[0])

    
    if len(sim_file_folders_list) > 0:
        model = OrcFxAPI.Model(sim_file_folders_list[0] + '\\' + sim_files[0]) 
    else:
        model = OrcFxAPI.Model(folders[0] + '\\' + sim_files[0])    
                    

    for key, value in extraction_data.items():
    
        print (key, value)
    
        object_name = value[1]
        OrcFxVariableName = value[0]
        objectExtra = value[2]
    
        extraction = st.OrcFxExtr.TH(model[object_name] ,OrcFxVariableName, \
                                       OrcFxAPI.SpecifiedPeriod(t_start_res, t_end_res), eval(objectExtra))
        print(key)
        print(extraction.getTHavg())
        print(extraction.getTHmax())
        print(extraction.getTHmin())
        print(extraction.getTHabsmax())
        print(extraction.getTHstddev())
        print(extraction.getTH())
        print(extraction.getUnit())

In [7]:
# Loop through folders, sim-files and data definitions and save to csv

for folder in folders:
    
    print(folder)
    
    # define some basic variables
    
    # Columns in statistical data extraction
    cols = ['Sim-file', 'Average','Max','Min','Abs max','std. dev']
    
    sim_files, sim_file_folders_list, prefix = st.folders_files.list_simfiles(folder)
    
    GenVariableNames = list(extraction_data.keys())
        
    base_df, TH_base_df = st.dfs.CreateDFs(sim_files, GenVariableNames, time_defs, cols)
    
    base_df


    # create strings for dataframes
    df_str = []
    for i in range(len(GenVariableNames)):
        print(prefix +'_[' + GenVariableNames[i] + ']')
        df_str.append(prefix +'_[' + GenVariableNames[i] + ']')
    
    # Create stat value dfs
    df = df_str.copy()
    for i in range(len(GenVariableNames)):
        df[i] = base_df.copy()
   
    
    for (idx, sim_file) in enumerate(sim_files, 1):
          
        TH_df = TH_base_df.copy()
        
        if len(sim_file_folders_list) > 0:
            model = OrcFxAPI.Model(sim_file_folders_list[sim_files.index(sim_file)] + '\\' + sim_file) 
        else:
            model = OrcFxAPI.Model(folder + '\\' + sim_file)    
    
        # Loop through parameters 

        # Extract data
        var_counter = 0
        for key, value in extraction_data.items():
    
            object_name = value[1]
            OrcFxVariableName = value[0]
            objectExtra = value[2]
            
            extraction = st.OrcFxExtr.TH(model[object_name] ,OrcFxVariableName, \
                            OrcFxAPI.SpecifiedPeriod(t_start_res, t_end_res), eval(objectExtra))
        
            #  Stat value extraction
            df[var_counter][cols[0]].loc[idx-1] = sim_file
            df[var_counter][cols[1]].loc[idx-1] = extraction.getTHavg()
            df[var_counter][cols[2]].loc[idx-1] = extraction.getTHmax()
            df[var_counter][cols[3]].loc[idx-1] = extraction.getTHmin()
            df[var_counter][cols[4]].loc[idx-1] = extraction.getTHabsmax()
            df[var_counter][cols[5]].loc[idx-1] = extraction.getTHstddev()
            df[var_counter].index.name = extraction.getUnit()
            #df[var_counter]
            
            var_counter += 1
            
            # TH extraction
            col = pd.concat([pd.Series([extraction.getUnit()]), pd.Series(extraction.getTH())], 
                axis = 0).reset_index(drop = True)
            
            TH_df[key] = col
    
        # Print TH values to csv 
        sim_file_str = os.path.splitext(sim_file)[0]
        TH_df.to_csv (folder + '\\' + sim_file_str  + '_TH.csv', header=True)          
            

        


D:\100 - Project files\02 - Orcaflex Runs\302 - IPC floating set 3 gain -10 sat 3deg


,Sim-file,Average,Max,Min,Abs max,std. dev
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
61,NaN,NaN,NaN,NaN,NaN,NaN
62,NaN,NaN,NaN,NaN,NaN,NaN
63,NaN,NaN,NaN,NaN,NaN,NaN
64,NaN,NaN,NaN,NaN,NaN,NaN


302_[RootMxb1]
302_[RootMyb1]
302_[RootMzb1]
302_[TipDxb1]
302_[TipDyb1]
302_[NcIMUTAxs]
302_[NcIMUTAys]
302_[NcIMUTAzs]
302_[YawBrTDxt]
302_[YawBrTDyt]
302_[TwrBsMxt]
302_[TwrBsMyt]
302_[PtfmSurge]
302_[PtfmHeave]
302_[PtfmPitch]
302_[FAIRTEN1]
302_[FAIRTEN2]
302_[FAIRTEN3]
302_[ANCHTEN1]
302_[ANCHTEN2]
302_[ANCHTEN3]
302_[Blade pitch1]
302_[Generator power]
302_[Generator torque]
302_[Rotor thrust]
302_[Rotor ang. vel.]
00001_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED10001)_W(NTM_U4.0_SEED10001).sim
00002_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED10002)_W(NTM_U4.0_SEED10002).sim
00003_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED10003)_W(NTM_U4.0_SEED10003).sim
00004_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED10004)_W(NTM_U4.0_SEED10004).sim
00005_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED10005)_W(NTM_U4.0_SEED10005).sim
00006_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED10006)_W(NTM_U4.0_SEED10006).sim
00007_w(Irreg_DIR000_Hs8.0_Tp12.7_U06_SEED10001)_W(NTM_U6.0_SEED10001).sim
00008_w(Irreg_DIR000_Hs8.0_Tp12.7_U06_SEED10002)_

In [8]:
    # Print stat values to csv        
    for i in range(len(GenVariableNames)):
        print(df_str[i])
        df[i].to_csv (folder + '\\' + df_str[i]  + '.csv', header=True)
        df[i]

302_[RootMxb1]


,Sim-file,Average,Max,Min,Abs max,std. dev
kN.m,,,,,,
0,00001_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,-8569.86,6677.05,-22552.5,22552.5,4480.36
1,00002_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,-9796.92,5367.23,-22019.7,22019.7,3923.01
2,00003_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,-8995.74,3129.47,-18462.7,18462.7,3605.05
3,00004_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,-9411.28,6780.73,-19646,19646,4134.65
4,00005_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,-8697.64,4699.3,-21821.2,21821.2,4318.59
...,...,...,...,...,...,...
61,00062_w(Irreg_DIR000_Hs9.8_Tp14.1_U24_SEED1000...,-13408.1,29985.6,-49805.7,49805.7,12248.7
62,00063_w(Irreg_DIR000_Hs9.8_Tp14.1_U24_SEED1000...,-12910.9,27486,-56380.7,56380.7,12680.6
63,00064_w(Irreg_DIR000_Hs9.8_Tp14.1_U24_SEED1000...,-15007.4,26262.3,-53822.4,53822.4,12633.7


302_[RootMyb1]


,Sim-file,Average,Max,Min,Abs max,std. dev
kN.m,,,,,,
0,00001_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,-388.338,18539.2,-19784.6,19784.6,12184.1
1,00002_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,-312.525,18514,-19193.7,19193.7,12145.5
2,00003_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,-387.537,17925.9,-18948.8,18948.8,12121.3
3,00004_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,-420.564,17833.8,-19587.3,19587.3,12201.4
4,00005_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,-319.146,17908.8,-18948.3,18948.3,12098.6
...,...,...,...,...,...,...
61,00062_w(Irreg_DIR000_Hs9.8_Tp14.1_U24_SEED1000...,-6108.21,25825.2,-40217.3,40217.3,13136.4
62,00063_w(Irreg_DIR000_Hs9.8_Tp14.1_U24_SEED1000...,-5801.41,30840.7,-37316.7,37316.7,13452.5
63,00064_w(Irreg_DIR000_Hs9.8_Tp14.1_U24_SEED1000...,-6262.85,25556.1,-37792.7,37792.7,13232.1


302_[RootMzb1]


...

,Sim-file,Average,Max,Min,Abs max,std. dev
kN.m,,,,,,
0,00001_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,-1014.52,-0,-3340.75,3340.75,836.642
1,00002_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,-1329.64,-0,-2883.05,2883.05,708.882
2,00003_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,-1010.01,-0,-2388.95,2388.95,466.746
3,00004_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,-1206.39,-0,-2778.52,2778.52,617.938
4,00005_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,-998.238,-21.8013,-3440.07,3440.07,811.274
...,...,...,...,...,...,...
61,00062_w(Irreg_DIR000_Hs9.8_Tp14.1_U24_SEED1000...,-19624,-19624,-19624,19624,0
62,00063_w(Irreg_DIR000_Hs9.8_Tp14.1_U24_SEED1000...,-19624,-19624,-19624,19624,0
63,00064_w(Irreg_DIR000_Hs9.8_Tp14.1_U24_SEED1000...,-19624,-19624,-19624,19624,0


302_[Rotor thrust]


,Sim-file,Average,Max,Min,Abs max,std. dev
kN,,,,,,
0,00001_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,210.677,620.193,-106.841,620.193,131.415
1,00002_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,262.341,562.188,-115.053,562.188,112.774
2,00003_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,224.727,480.931,-19.1789,480.931,91.1745
3,00004_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,245.211,563.372,-108.128,563.372,98.5537
4,00005_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,216.498,588.734,-58.2079,588.734,118.186
...,...,...,...,...,...,...
61,00062_w(Irreg_DIR000_Hs9.8_Tp14.1_U24_SEED1000...,785.484,1452.42,117.351,1452.42,184.282
62,00063_w(Irreg_DIR000_Hs9.8_Tp14.1_U24_SEED1000...,788.212,1555.81,157.382,1555.81,183.764
63,00064_w(Irreg_DIR000_Hs9.8_Tp14.1_U24_SEED1000...,807.455,1523.58,252.191,1523.58,203.671


302_[Rotor ang. vel.]


,Sim-file,Average,Max,Min,Abs max,std. dev
RPM,,,,,,
0,00001_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,4.95478,5.22634,4.56625,5.22634,0.124821
1,00002_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,4.999,5.20698,4.83426,5.20698,0.0660272
2,00003_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,4.99746,5.15837,4.85707,5.15837,0.0493086
3,00004_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,4.99843,5.17919,4.85199,5.17919,0.0535079
4,00005_w(Irreg_DIR000_Hs6.3_Tp11.5_U04_SEED1000...,5.00173,5.19297,4.83773,5.19297,0.0521393
...,...,...,...,...,...,...
61,00062_w(Irreg_DIR000_Hs9.8_Tp14.1_U24_SEED1000...,7.56647,9.026,5.5355,9.026,0.521568
62,00063_w(Irreg_DIR000_Hs9.8_Tp14.1_U24_SEED1000...,7.53432,9.44644,6.44981,9.44644,0.436695
63,00064_w(Irreg_DIR000_Hs9.8_Tp14.1_U24_SEED1000...,7.53953,8.91695,6.15844,8.91695,0.54838
